In [1]:
#Create Folder for modelling checkpoint
import os
import pandas as pd
# pickle to save dictionary in file
import pickle 
cwd = os.path.normpath(os.getcwd()+ os.sep + os.pardir + os.sep + os.pardir)

In [91]:
def loadResultDictionary(cwd, dictinaryName):
    with open(cwd + f'/results/{dictinaryName}.pkl', 'rb') as f:
        dictionary = pickle.load(f)
    return dictionary

def getMetricsForClientInCluster(dictionary, model, horizon, feature, cluster, client, mse, rmse, mape, mae, time): 
    mse = mse + dictionary['Federated'][model][horizon][feature][cluster][client]['MSE']
    rmse = rmse+ dictionary['Federated'][model][horizon][feature][cluster][client]['RMSE']
    mape = mape + dictionary['Federated'][model][horizon][feature][cluster][client]['MAPE']
    mae = mae + dictionary['Federated'][model][horizon][feature][cluster][client]['MAE']
    time = time + dictionary['Federated'][model][horizon][feature][cluster][client]['Time']
    
    return mse, rmse, mape, mae, time

def getMetricsForCluster(cluster_length, mse, rmse, mape, mae, time):
    mse = mse/cluster_length
    rmse = rmse /cluster_length
    mape = mape/cluster_length
    mae = mae/cluster_length
    time = time/cluster_length
    return mse, rmse, mape, mae, time

def namestr(obj, namespace):
    """
    Get the name of the variable as a String
    """
    return [name for name in namespace if namespace[name] is obj][0]

def getResultsFromFederatedDictionary(dictionaries): 
    #Create dataframe with Coumns(Szenario, MSE, RMSE, MAPE, MAE, Time)
    result_df = pd.DataFrame(columns=['Learning', 'Horizon', 'Feature', 'Model', 'Cluster','MSE', 'RMSE', 'MAPE', 'MAE', 'Time'])
    
    for dictionary in dictionaries:
        horizon = namestr(dictionary, globals()).split('_')[1]
        feature = namestr(dictionary, globals()).split('_')[2]

        # For each of the models LSTM, CNN, Transformer
        for model in dictionary['Federated']:

            #For each of the clusters 0-5
            for cluster in dictionary['Federated'][model][horizon][feature]:
                #print("Model: ", model, "h: ", horizon, "f: ", feature)
                
                #initilize cluster_length and metrics
                cluster_length, mse, rmse, mape, mae, time = 0,0,0,0,0,0

                #Get the metrics for each client and add them up
                for idx, client in enumerate(dictionary['Federated'][model][horizon][feature][cluster]):
                    cluster_length = cluster_length + 1
                    mse, rmse, mape, mae, time = getMetricsForClientInCluster(
                        dictionary, model, horizon, feature,
                        cluster, client, mse, rmse, mape, mae, time
                    )

                #Devide the metrics by the number of clients in cluster to get average
                mse, rmse, mape, mae, time = getMetricsForCluster(cluster_length, mse, rmse, mape, mae, time)
                #Add metrics to the last row of the result_df
                result_df.loc[len(result_df)]= ['Federated', horizon, feature, model, cluster, mse, rmse, mape, mae, time]
            
    return result_df

In [92]:
#Load all 4 federated results dictionaries
fl_H12_F7 = loadResultDictionary(cwd, 'Federated_results_H12_F7')
fl_H24_F7 = loadResultDictionary(cwd, 'Federated_results_H24_F7')
fl_H12_F5 = loadResultDictionary(cwd, 'Federated_results_H12_F5')
fl_H24_F5 = loadResultDictionary(cwd, 'Federated_results_H24_F5')

df = getResultsFromFederatedDictionary([fl_H12_F7, fl_H24_F7, fl_H12_F5, fl_H24_F5])

df.head(20)

,Learning,Horizon,Feature,Model,Cluster,MSE,RMSE,MAPE,MAE,Time
0,Federated,H12,F7,LSTM,0,0.041943,0.204211,8.718837e+07,0.183136,2.647967
1,Federated,H12,F7,LSTM,1,0.019745,0.126301,4.923419e+07,0.084764,7.542402
2,Federated,H12,F7,LSTM,2,0.017669,0.127418,4.637050e+07,0.095374,7.253827
3,Federated,H12,F7,LSTM,3,0.055643,0.233891,1.047805e+08,0.210342,5.079003
4,Federated,H12,F7,LSTM,4,0.027356,0.165164,7.988741e+07,0.129357,6.461831
5,Federated,H12,F7,LSTM,5,0.023359,0.148722,6.309520e+07,0.100014,8.096773
6,Federated,H12,F7,CNN,0,0.047317,0.216986,1.041335e+08,0.196263,0.277498
7,Federated,H12,F7,CNN,1,0.027968,0.146355,6.144285e+07,0.099853,0.312652
8,Federated,H12,F7,CNN,2,0.025221,0.156726,7.402277e+07,0.130403,0.343698
9,Federated,H12,F7,CNN,3,0.067487,0.258435,1.248741e+08,0.230147,0.369721


In [93]:
df2 = df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()
df2


C:\Users\jonas\AppData\Local\Temp\ipykernel_996\1450212595.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = df.groupby(['Horizon','Feature', 'Model'])['MSE', 'RMSE', 'MAPE', 'MAE', 'Time'].sum()


MSE      RMSE          MAPE       MAE  \
Horizon Feature Model                                                     
H12     F5      CNN          0.280923  1.246526  6.112839e+08  1.050888   
                LSTM         0.186033  1.004272  4.323545e+08  0.802181   
                Transformer  0.185323  1.003066  4.336362e+08  0.804882   
        F7      CNN          0.299883  1.280678  6.684458e+08  1.081970   
                LSTM         0.185716  1.005708  4.305561e+08  0.802988   
                Transformer  0.185602  1.003962  4.322320e+08  0.809917   
H24     F5      CNN          0.287227  1.254387  6.179984e+08  1.063341   
                LSTM         0.192366  1.024861  4.552913e+08  0.837050   
                Transformer  0.192973  1.026971  4.604955e+08  0.847942   
        F7      CNN          0.275966  1.234907  6.130923e+08  1.049024   
                LSTM         0.196303  1.036555  4.671387e+08  0.850154   
                Transformer  0.193491  1.028433  4.493285e+08  0.842171   

                                  Time  
Horizon Feature Model                   
H12     F5      CNN           3.175604  
                LSTM         50.387979  
                Transformer  15.414098  
        F7      CNN           2.019861  
                LSTM         37.081803  
                Transformer  18.495457  
H24     F5      CNN           3.835397  
                LSTM         53.728168  
                Transformer  31.714660  
        F7      CNN           2.815636  
                LSTM         49.194875  
                Transformer  22.651760